In [1]:
from stellargraph.mapper import CorruptedFullBatchNodeGenerator
from stellargraph import StellarGraph
from stellargraph.layer import GCNInfoMax
from stellargraph import datasets

import random
import numpy as np
from sklearn import preprocessing, feature_extraction, model_selection
from sklearn.linear_model import LogisticRegression
from IPython.display import display, HTML

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense
import tensorflow as tf
from tensorflow.keras import Model

In [2]:
def info_loss(y_true, y_pred):
    
    return -tf.math.reduce_mean(tf.math.log(y_pred))

In [3]:
dataset = datasets.Cora()
display(HTML(dataset.description))
G, node_subjects = dataset.load()

In [4]:
train_subjects, test_subjects = model_selection.train_test_split(
    node_subjects, train_size=0.1, test_size=None, stratify=node_subjects
)

target_encoding = preprocessing.LabelBinarizer()

train_targets = target_encoding.fit_transform(train_subjects)
test_targets = target_encoding.transform(test_subjects)

In [5]:
generator = CorruptedFullBatchNodeGenerator(G,)
gen = generator.flow(G.nodes(),)

infomax = GCNInfoMax(
    [512,], 
    activations=[tf.keras.layers.PReLU(),], 
    generator=generator, 
    bias=True,
)

x_in, x_out = infomax.unsupervised_node_model()

model = Model(inputs=x_in, outputs=x_out)
model.compile(loss=tf.nn.sigmoid_cross_entropy_with_logits, optimizer=Adam(lr=1e-3))

Using GCN (local pooling) filters...


In [10]:
model.fit(gen, epochs=20)

  ...
    to  
  ['...']
Train for 1 steps
Epoch 1/20
1/1 [==============================] - 0s 146ms/step - loss: 0.7475
Epoch 2/20
1/1 [==============================] - 0s 125ms/step - loss: 0.7443
Epoch 3/20
1/1 [==============================] - 0s 113ms/step - loss: 0.7371
Epoch 4/20
1/1 [==============================] - 0s 117ms/step - loss: 0.7280
Epoch 5/20
1/1 [==============================] - 0s 118ms/step - loss: 0.7188
Epoch 6/20
1/1 [==============================] - 0s 112ms/step - loss: 0.7106
Epoch 7/20
1/1 [==============================] - 0s 117ms/step - loss: 0.7040
Epoch 8/20
1/1 [==============================] - 0s 117ms/step - loss: 0.6992
Epoch 9/20
1/1 [==============================] - 0s 117ms/step - loss: 0.6961
Epoch 10/20
1/1 [==============================] - 0s 125ms/step - loss: 0.6944
Epoch 11/20
1/1 [==============================] - 0s 127ms/step - loss: 0.6937
Epoch 12/20
1/1 [==============================] - 0s 124ms/step - loss: 0.6938
Epoch 

In [11]:
x_emb_in, x_emb_out = infomax.embedding_model(model)
emb_model = Model(inputs=x_emb_in, outputs=x_emb_out)

In [12]:
test_gen = generator.flow(test_subjects.index)
train_gen = generator.flow(train_subjects.index)

test_embeddings = emb_model.predict(test_gen)
train_embeddings = emb_model.predict(train_gen)

lr = LogisticRegression()
lr.fit(train_embeddings, train_subjects)

y_pred = lr.predict(test_embeddings)
(y_pred == test_subjects).mean()

/Users/kieranricardo/anaconda3/envs/tf2-stellar/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/kieranricardo/anaconda3/envs/tf2-stellar/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.32198523379819527

In [13]:
tf.math.sigmoid(model.predict(test_gen)).numpy().min(), tf.math.sigmoid(model.predict(test_gen)).numpy().max()

(0.372261, 0.50547016)

# END OF DEMO